<a href="https://colab.research.google.com/github/rikanga/Easy-Numpy/blob/main/UncertainEstimatesClassifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Estimaton de l'incertitude dans le modèle**

In [1]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_blobs, make_circles
from sklearn.model_selection import train_test_split

In [2]:
X, y = make_circles(noise=.25, factor=0.5, random_state=1)

In [6]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt

In [5]:
y_named = np.array(['blue', 'red'])[y]

In [7]:
!pip install mglearn

     |████████████████████████████████| 540 kB 4.9 MB/s 
  Created wheel for mglearn: filename=mglearn-0.1.9-py2.py3-none-any.whl size=582639 sha256=797c30dc627e2d9fb2ef01f5358ba37226430d182e5f8e417b7c30a8095f5d50
  Stored in directory: /root/.cache/pip/wheels/f1/17/e1/1720d6dcd70187b6b6c3750cb3508798f2b1d57c9d3214b08b
Successfully built mglearn


In [9]:
X_train, X_test, y_train_named, y_test_named, y_train, y_test = train_test_split(X, y_named, y, random_state=0)

In [10]:
y_train_named.shape, y_train.shape

((75,), (75,))

In [11]:
y_train_named[:10]

array(['blue', 'blue', 'blue', 'blue', 'blue', 'blue', 'blue', 'blue',
       'blue', 'red'], dtype='<U4')

In [12]:
y_train[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

**Model**

In [13]:
model = GradientBoostingClassifier(random_state=0)
model.fit(X_train, y_train_named)

GradientBoostingClassifier(random_state=0)

In [19]:
X_test.shape

(25, 2)

**Voyons le 10 premiers entrées**

In [23]:
model.decision_function(X_test)[:10]

array([ 4.13592603, -1.70169917, -3.95106099, -3.62609552,  4.28986642,
        3.66166081, -7.69097179,  4.11001686,  1.10753937,  3.40782222])

### **Découvrir les prédictions**


*   Nous pouvons découvrir les prédictions juste grâce au signe sur les données que renvoie **decision_function()**
*   Du coup nous pouvons voir que **True** représente les données de la classe 1, donc leurs prédictions et **False** représente les données de la **classe 0**



In [24]:
model.decision_function(X_test) > 0

array([ True, False, False, False,  True,  True, False,  True,  True,
        True, False,  True,  True, False,  True, False, False, False,
        True,  True,  True,  True,  True, False, False])

In [27]:
print("Decision function:\n")
print(model.decision_function(X_test) > 0)

print('\nPredictions:\n')
print(model.predict(X_test))

Decision function:

[ True False False False  True  True False  True  True  True False  True
  True False  True False False False  True  True  True  True  True False
 False]

Predictions:

['red' 'blue' 'blue' 'blue' 'red' 'red' 'blue' 'red' 'red' 'red' 'blue'
 'red' 'red' 'blue' 'red' 'blue' 'blue' 'blue' 'red' 'red' 'red' 'red'
 'red' 'blue' 'blue']
